In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
path = "./results/results-mcs_compression_table-2024-05-21_08-03-23.csv"
df = pd.read_csv(path, sep=";")
df

In [ ]:
df["duration_ns"].max() / (60 * 10**9)

In [ ]:
df["tid"] = df["taskset_position"]
df["schedulable"] = df["is_safe"]
df["Schedulable"] = df["is_safe"]
df["n_task"] = df["nbt"]
df["Number of tasks"] = df["nbt"]
df["duration_s"] = df["duration_ns"] / 1e9
df["n_visited"] = df["visited_count"]
df["search_type"] = df["use_idlesim"].apply(lambda x: "ACBFS" if x else "BFS")

df["safe_oracles_fmt"] = df["safe_oracles"].str.strip("[']")
df["unsafe_oracles_fmt"] = df["unsafe_oracles"].str.strip("[']")
df["safe_oracles_fmt"] = df["safe_oracles_fmt"].apply(lambda x: x if len(x) else None)
df["unsafe_oracles_fmt"] = df["unsafe_oracles_fmt"].apply(
    lambda x: x if len(x) else None
)

df["oracle"] = df.apply(
    lambda x: (
        x["safe_oracles_fmt"]
        if x["safe_oracles_fmt"]
        else x["unsafe_oracles_fmt"] if x["unsafe_oracles_fmt"] else "none"
    ),
    axis=1,
)

df["unsafe"] = df["oracle"] == "hi-over-demand"
df["unsafe"] = df["unsafe"].apply(lambda x: "hi-over-demand" if x else None)

In [ ]:
scope = df["search_type"] == "BFS"
scope &= df["unsafe"].isna()
df_ref = df.loc[scope, ["tid", "duration_s", "n_visited"]].rename(
    columns={"duration_s": "duration_s_ref", "n_visited": "n_visited_ref"}
)
df = df.merge(df_ref, on="tid")

In [ ]:
df["n_visited_norm"] = df["n_visited"]
df["duration_s_norm"] = df["duration_s"]

df.loc[~scope, "n_visited_norm"] = (
    df.loc[~scope, "n_visited"] / df.loc[~scope, "n_visited_ref"] - 1
) * 100
df.loc[~scope, "duration_s_norm"] = (
    df.loc[~scope, "duration_s"] / df.loc[~scope, "duration_s_ref"] - 1
) * 100

In [ ]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [ ]:
df

In [ ]:
df["Search"] = df["search_type"]
df["Oracle"] = df["unsafe"].apply(
    lambda x: "HI over demand" if x == "hi-over-demand" else "None"
)

In [ ]:
s = df["n_visited_norm"].idxmin()
tsp = df.loc[s, "taskset_position"]
df.loc[
    df["taskset_position"] == tsp,
    [
        "use_case",
        "is_safe",
        "visited_count",
        "n_visited_norm",
        "duration_s",
        "duration_s_norm",
    ],
]

In [ ]:
dfagg = (
    df.groupby(["Search", "Oracle"], dropna=False)[
        ["n_visited_norm", "duration_s_norm"]
    ]
    .agg(["min", "mean", "std", "median", "max"])
    .apply(lambda x: round(x, 2))
    .applymap(str)
)

In [ ]:
dfagg.columns = dfagg.columns.set_levels(
    ["Number of visited states before halt", "Execution time"], level=0
)

In [ ]:
dfagg = dfagg.reindex(["BFS", "ACBFS"], level=0)
dfagg = dfagg.reindex(["None", "HI over demand"], level=1)

In [ ]:
dfagg.iloc[1:] = " (" + dfagg.iloc[1:] + "\%)"

In [ ]:
dfagg2_int = (
    df.groupby(["Search", "Oracle"], dropna=False)[["n_visited"]]
    .agg(["min", "mean", "std", "median", "max"])
    .applymap(lambda x: round(x))
)

dfagg2_int = dfagg2_int.reindex(["BFS", "ACBFS"], level=0)
dfagg2_int = dfagg2_int.reindex(["None", "HI over demand"], level=1)

dfagg2_int = dfagg2_int.T

dfagg2 = dfagg2_int.applymap(str)


dfagg2

In [ ]:
(1 - 29 / 9905) * 100

In [ ]:
r = dfagg2_int.iloc[:, 0]
for i in range(3):
    dfagg2_int.iloc[:, 1 + i] /= r
    dfagg2_int.iloc[:, 1 + i] -= 1

In [ ]:
(dfagg2_int * 100).applymap(lambda x: round(x))